In [3]:
import os
os.chdir('../')
print(os.getcwd())

/DATA/rahul_1911mt11/multi-tasking/EEG-Multi-Tasking


In [4]:
import torch 
import random
import traceback
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

In [7]:
# Device configuration
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
episodes = 100
num_epochs = 1
num_classes = 2
batch_size = 50
lr1 = 0.000001
lr2 = 0.001

In [8]:
from src.dataset import single_task_dataset
from src.dataset import multi_task_dataset

In [9]:
df = pd.read_csv('input/data.csv')

In [10]:
df = df.sample(frac=1).reset_index(drop=True)
df.shape

(3480, 5)

In [11]:
def conv_block(in_channels, out_channels):
    '''Convolution Block of 3x3 kernels + batch norm + maxpool of 2x2'''

    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 5, padding=2),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    


In [12]:
class TaskNet(nn.Module):
    def __init__(self):
        super(TaskNet, self).__init__()

        self.conv1 = conv_block(1, 20)
        self.conv2 = conv_block(20, 40)
        self.conv3 = conv_block(40, 60)

        self.head = nn.Linear(9 * 4 * 60, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        # Features of shape (batch_size, 64)
        feat = x.reshape(x.size(0), -1)

        # Output
        out = self.head(feat)

        return out

In [13]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        self.conv1 = conv_block(1, 20)
        self.conv2 = conv_block(20, 40)
        self.conv3 = conv_block(40, 60)
        self.head1 = nn.Linear(9 * 4 * 60 * 2, 100)
        self.head2 = nn.Linear(100, 50)
        self.head3 = nn.Linear(50, 20)
        self.head4 = nn.Linear(20, 2)
        
        task_model = TaskNet().to(device)
        try:
            checkpoint = torch.load('models/adversarial_train_batch/tasknet.pth', map_location=torch.device('cuda:3' if torch.cuda.is_available() else 'cpu'))
            task_model.load_state_dict(checkpoint['weights'])
#             print("--------- Successfully loaded previous model weights --------------------")
        except:
#             traceback.print_exc()
            print("-------Creating task net from scratch -----------")
        self.conv11 = task_model.conv1
        self.conv22 = task_model.conv2
        self.conv33 = task_model.conv3


    def forward(self, x):
        
        x1 = self.conv1(x)
        x1 = self.conv2(x1)
        x1 = self.conv3(x1)
        
        x2 = self.conv11(x)
        x2 = self.conv22(x2)
        x2 = self.conv33(x2)
        
        

        # Features of shape (batch_size, 64)
        feat1 = x1.reshape(x1.size(0), -1)
        feat2 = x2.reshape(x2.size(0), -1)
        feat  = torch.cat((feat1, feat2),1)
  
        # Output
        out = self.head1(feat)
        out = F.relu(out)
        out = self.head2(out)
        out = F.relu(out)
        out = self.head3(out)
        out = F.relu(out)
        out = self.head4(out)

        return out

In [16]:
# Train the model

for episode in range(episodes):
    
    n = random.randint(1,9)

    df_train = df.loc[(df.Type=='train') & (df.Subject!=10)].reset_index(drop = True)
    train_dataset = single_task_dataset(df_train)
    train_dataset = multi_task_dataset(df_train)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size, 
                                               shuffle=True)
    
    model1 = ConvNet().to(device)
    try:
        checkpoint = torch.load('models/adversarial_train_batch/convnet.pth', map_location=torch.device('cuda:3' if torch.cuda.is_available() else 'cpu'))
        model1.load_state_dict(checkpoint['weights'])
#         print("--------- Successfully loaded previous model weights --------------------")
    except:
        print("-------Creating convnet from scratch -----------")
    
    model2 = TaskNet().to(device)
    try:
        checkpoint = torch.load('models/adversarial_train_batch/tasknet.pth', map_location=torch.device('cuda:3' if torch.cuda.is_available() else 'cpu'))
        model2.load_state_dict(checkpoint['weights'])
#         print("--------- Successfully loaded previous model weights --------------------")
    except:
        print("-------Creating task net from scratch -----------")
        

    # Loss and optimizer
    criterion1 = nn.CrossEntropyLoss()
    optimizer1 = torch.optim.Adam(model1.parameters(), lr=lr1)

    criterion2 = nn.CrossEntropyLoss()
    optimizer2 = torch.optim.Adam(model2.parameters(), lr=lr2)
    


    total_step = len(train_loader)
    for epoch in range(num_epochs):


        for i, (images, labels, subjects) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            subjects = subjects.to(device)

            # Forward pass 1
            outputs1 = model1(images)
            loss1 = criterion1(outputs1, labels)



            # Forward pass 2
            outputs2 = model2(images)
            loss2 = criterion2(outputs2, subjects)

            # Backward and optimize 1
            optimizer1.zero_grad()
            loss1.backward()
            optimizer1.step()

            # Backward and optimize 2
#             optimizer2.zero_grad()
#             loss2.backward()
#             optimizer2.step()

            torch.save({'weights': model2.state_dict()},'models/adversarial_train_batch/tasknet.pth')
            torch.save({'weights': model1.state_dict()},'models/adversarial_train_batch/convnet.pth')

            if (i+1) % 10000 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, i+1, total_step, loss1.item()))
    print(f"-------------  EPISODE : {episode}  ------------")
    print("------------------------------------------------")
    train_accuracy_model1()
    test_accuracy_model1()
    train_accuracy_model2()
    test_accuracy_model2()
    print("------------------------------------------------", '\n')

-------------  EPISODE : 0  ------------
------------------------------------------------
Train Accuracy: 76.55797101449275 %
Test Accuracy: 76.38888888888889 %
Train Accuracy for tasknet: 99.34782608695652 %
Test Accuracy for tasknet: 81.80555555555556 %
------------------------------------------------ 

-------------  EPISODE : 1  ------------
------------------------------------------------
Train Accuracy: 76.52173913043478 %
Test Accuracy: 75.97222222222223 %
Train Accuracy for tasknet: 99.23913043478261 %
Test Accuracy for tasknet: 81.66666666666667 %
------------------------------------------------ 

-------------  EPISODE : 2  ------------
------------------------------------------------
Train Accuracy: 76.52173913043478 %
Test Accuracy: 76.25 %
Train Accuracy for tasknet: 99.27536231884058 %
Test Accuracy for tasknet: 81.38888888888889 %
------------------------------------------------ 

-------------  EPISODE : 3  ------------
------------------------------------------------
T

In [1]:
# Test the model

def test_accuracy_model1():
    
    df_test = df.loc[(df.Type=='test') & (df.Subject!=10)].reset_index(drop = True)
    test_dataset = multi_task_dataset(df_test)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                               batch_size=1, 
                                               shuffle=True)
    
    model1 = ConvNet().to(device)
    try:
        checkpoint = torch.load('models/adversarial_train_batch/convnet.pth', map_location=torch.device('cuda:3' if torch.cuda.is_available() else 'cpu'))
        model1.load_state_dict(checkpoint['weights'])
    #         print("--------- Successfully loaded previous model weights --------------------")
    except:
        print("-------Creating convnet from scratch -----------")
    

    model1.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels, subjects in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model1(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Test Accuracy: {} %'.format(100 * correct / total))


# Test the model

def train_accuracy_model1():
    
    df_train = df.loc[(df.Type=='train') & (df.Subject!=10)].reset_index(drop = True)
    train_dataset = multi_task_dataset(df_train)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=1, 
                                               shuffle=True)
    
    model1 = ConvNet().to(device)
    try:
        checkpoint = torch.load('models/adversarial_train_batch/convnet.pth', map_location=torch.device('cuda:3' if torch.cuda.is_available() else 'cpu'))
        model1.load_state_dict(checkpoint['weights'])
    #         print("--------- Successfully loaded previous model weights --------------------")
    except:
        print("-------Creating convnet from scratch -----------")

    model1.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels, subjects in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model1(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Train Accuracy: {} %'.format(100 * correct / total))

In [2]:
# Test the model
def test_accuracy_model2():
    
    df_test = df.loc[(df.Type=='test') & (df.Subject!=10)].reset_index(drop = True)
    test_dataset = multi_task_dataset(df_test)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                               batch_size=1, 
                                               shuffle=True)
    
    model2 = TaskNet().to(device)
    try:
        checkpoint = torch.load('models/adversarial_train_batch/tasknet.pth', map_location=torch.device('cuda:3' if torch.cuda.is_available() else 'cpu'))
        model2.load_state_dict(checkpoint['weights'])
    #         print("--------- Successfully loaded previous model weights --------------------")
    except:
        print("-------Creating task net from scratch -----------")
    
    
    model2.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels, subjects in test_loader:
            images = images.to(device)
            labels = subjects.to(device)
            outputs = model2(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
#             print(predicted.item(), labels.item())

    print('Test Accuracy for tasknet: {} %'.format(100 * correct / total))

# Test the model
def train_accuracy_model2():
    
    df_train = df.loc[(df.Type=='train') & (df.Subject!=10)].reset_index(drop = True)
    train_dataset = multi_task_dataset(df_train)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=1, 
                                               shuffle=True)
    
    model2 = TaskNet().to(device)
    try:
        checkpoint = torch.load('models/adversarial_train_batch/tasknet.pth', map_location=torch.device('cuda:3' if torch.cuda.is_available() else 'cpu'))
        model2.load_state_dict(checkpoint['weights'])
    #         print("--------- Successfully loaded previous model weights --------------------")
    except:
        print("-------Creating task net from scratch -----------")
    
    
    model2.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels, subjects in train_loader:
            images = images.to(device)
            labels = subjects.to(device)
            outputs = model2(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
#             print(predicted.item(), labels.item())

    print('Train Accuracy for tasknet: {} %'.format(100 * correct / total))

In [ ]:
train_accuracy_model1()
test_accuracy_model1()
train_accuracy_model2()
test_accuracy_model2()

In [ ]:
train_accuracy_model2()

In [ ]:
test_accuracy_model2()